In [0]:
# Power BI export (CE-friendly): download CSVs from the grid via display(df)
# Inputs: fx_impact.gold_* tables
# Outputs: CSV downloads you’ll save locally (no FileStore / DBFS browsing needed)


In [0]:
# Cell 1 — Params (toggle complete months)
from pyspark.sql import functions as F

FILTER_TO_COMPLETE_MONTHS = True  # set False if you want current partial month included

# compute last complete month end
last_complete = spark.sql("SELECT last_day(add_months(current_date(), -1)) AS lcd").collect()[0]["lcd"]
print("Last complete month:", last_complete)


In [0]:
# Cell 2 — Sanity checks (gold tables exist)
tables = [
  "fx_impact.gold_monthly_metrics",
  "fx_impact.gold_monthly_totals",
  "fx_impact.gold_period_summary",
  "fx_impact.gold_covid_period_kpis_by_cmd"
]
for t in tables:
    spark.sql(f"DESCRIBE TABLE {t}").show(truncate=False)


In [0]:
# Cell 3 — Main fact export (metrics)
where_clause = f"WHERE month <= DATE('{last_complete}')" if FILTER_TO_COMPLETE_MONTHS else ""
q = f"""
SELECT month, cmdCode, 
       cmdDesc,            -- short (for visuals)
       cmdDesc_long,       -- long (for tooltips)
       covid_period,
       import_usd, import_eur,
       USD_per_EUR, JPY_per_EUR, CNY_per_EUR,
       share_of_total_eur, eur_mom_pct, eur_yoy_pct,
       eur_vol_3m, eur_vol_6m, fx_mom_pct, fx_vol_3m, fx_vol_6m
FROM fx_impact.gold_monthly_metrics
{where_clause}
ORDER BY month, cmdCode
"""
df_metrics = spark.sql(q)
display(df_metrics)  # ⬅ Click Download → CSV (name it gold_monthly_metrics.csv)


In [0]:
# Cell 4 — Totals export (for fast cards/lines)
q = f"""
SELECT month, total_import_usd, total_import_eur, avg_usd_per_eur
FROM fx_impact.gold_monthly_totals
{where_clause}
ORDER BY month
"""
df_totals = spark.sql(q)
display(df_totals)  # Download → gold_monthly_totals.csv


In [0]:
# Cell 5 — COVID period summary (overall)
df_period = spark.sql("SELECT * FROM fx_impact.gold_period_summary")
display(df_period)  # Download → gold_period_summary.csv


In [0]:
# Cell 6 — COVID KPIs by commodity
df_bycmd = spark.sql("""
SELECT cmdCode, 
       cmdDesc,            -- short (for visuals)
       cmdDesc_long,       -- long (for tooltips)
       pre_eur, during_eur, post_eur,
       abs_change_during_vs_pre, pct_change_during_vs_pre,
       abs_change_post_vs_pre,   pct_change_post_vs_pre
FROM fx_impact.gold_covid_period_kpis_by_cmd
ORDER BY abs_change_post_vs_pre DESC NULLS LAST
""")
display(df_bycmd)  # Download → gold_covid_period_kpis_by_cmd.csv
